In [1]:
import os
import csv
import re
from nltk import sent_tokenize

import sys
sys.path.insert(0, '../')

from TextLabeller import *
from utils import read_list, normal_distribution, get_scatter_plot, normalize, inverse_distances, normed_dists_to_sims

In [3]:
DATA_FILE = "../data/emnlp_news.txt"
TEST_FILE = "../data/test_emnlp.txt"
GENERATED_FILE = "../data/generated_text2.txt"
METRIC = 'levenshtein'

# save these sentences and novelties to save computation time
# corpus_sentences = get_sentences(DATA_FILE)  # 304222 sentences
real_sentences = get_sentences(TEST_FILE) # 10785 sentences
fake_sentences = get_sentences(GENERATED_FILE)  # 11055 sentences

# Read the metrics results from the file
lev_real_diversities_distances = read_list(f'{METRIC}_diversities_real.txt')
lev_real_novelties_distances = read_list(f'{METRIC}_novelties_real.txt')
lev_fake_diversities_distances = read_list(f'{METRIC}_diversities_fake2.txt')
lev_fake_novelties_distances = read_list(f'{METRIC}_novelties_fake2.txt')

def _norm(dists):
    return [1 - dist for dist in dists]

lev_normed_real_diversities = lev_real_diversities_distances
lev_normed_real_novelties = lev_real_novelties_distances
lev_normed_fake_diversities = _norm(lev_fake_diversities_distances)
lev_normed_fake_novelties = _norm(lev_fake_novelties_distances)

# # Transform distances to similarities
# lev_normed_real_diversities = normed_dists_to_sims(normalize(lev_real_diversities_distances))
# lev_normed_real_novelties = normed_dists_to_sims(normalize(lev_real_novelties_distances))
# lev_normed_fake_diversities = normed_dists_to_sims(normalize(lev_fake_diversities_distances))
# lev_normed_fake_novelties = normed_dists_to_sims(normalize(lev_fake_novelties_distances))

# Ensure that number of sentences have same number of novelties and diversities
assert(len(fake_sentences) == len(lev_normed_fake_diversities) == len(lev_normed_fake_novelties))
assert(len(real_sentences) == len(lev_normed_real_diversities) == len(lev_normed_real_novelties))

real_dict = create_labelled_dictionary(real_sentences, lev_normed_real_novelties, lev_normed_real_diversities, 1)
write_to_csv(real_dict, filename=f'{METRIC}_labelled_real_text.csv')

# label fake text with 0
fake_dict = create_labelled_dictionary(fake_sentences, lev_normed_fake_novelties, lev_normed_fake_diversities, 0)
write_to_csv(fake_dict, filename=f'{METRIC}_labelled_fake_text.csv')